In [48]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [55]:
# 使用sk digit数据
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

# layer
def add_layer(inputs, in_size, out_size, layer_name, activation_function=None,keep_prob=1.0):
    
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]),name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size])+0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights), biases)
        with tf.name_scope('drop_out'):
            Wx_plus_b = tf.nn.dropout(Wx_plus_b,keep_prob)    
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.histogram_summary(layer_name+'/outputs', outputs)
        return outputs

#define 
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 64])  # 8x8
ys = tf.placeholder(tf.float32, [None, 10])

# add output layer
l1 = add_layer(xs, 64, 50, layer_name='l1',activation_function=tf.nn.tanh,keep_prob=keep_prob)
prediction = add_layer(l1,50,10,layer_name='l2',activation_function=tf.nn.softmax,keep_prob=1.0)

#交叉熵
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))  # loss
tf.scalar_summary('loss', cross_entropy)
# trian
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#=================run===============================
sess = tf.Session()
merged = tf.merge_all_summaries() # 将要记录的信息merge
# writer
train_writer  = tf.train.SummaryWriter("logs/train",sess.graph)
test_writer  = tf.train.SummaryWriter("logs/test",sess.graph)

# 2017-03-02 if using tensorflow >= 0.12
sess.run(tf.initialize_all_variables())



In [56]:
for i in range(500):
    # here to determine the keeping probability
    sess.run(train_step, feed_dict={xs: X_train, ys: y_train, keep_prob: 0.5})
    if i % 50 == 0:
        # record loss 记录信息
        train_result = sess.run(merged, feed_dict={xs: X_train, ys: y_train, keep_prob:0.5})
        test_result = sess.run(merged, feed_dict={xs: X_test, ys: y_test, keep_prob: 0.5})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_12' with dtype float
	 [[Node: Placeholder_12 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder_12', defined at:
  File "/home/clz/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/clz/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/clz/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/clz/anaconda2/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/clz/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/clz/anaco